## 🎯 이 노트북이 하는 일

### 파인튜닝(Fine-tuning)이란?
이미 학습된 AI 모델을 **내 데이터로 추가 학습**시켜 특정 분야에 더 잘하게 만드는 것

**비유**: 
- 의대 졸업생 (기본 모델) → 특정 병원에서 수련 (파인튜닝) → 그 병원 전문의 (특화 모델)
- 영어 선생님 (기본 모델) → 법률 문서로 추가 학습 (파인튜닝) → 법률 영어 전문가 (특화 모델)

### 우리의 목표
| Before | After |
|--------|-------|
| qwen3:8b (범용 한국어 모델) | qwen3-hr (HR 전문가 모델) |
| "개발팀 급여" → 가끔 틀린 SQL | "개발팀 급여" → 정확한 SQL |
| 회사 규정 모름 | 회사 규정 학습됨 |

### 전체 과정 (약 40분)
```
1. 환경 설정 (5분) - 필요한 라이브러리 설치
2. 데이터 로드 (1분) - 학습용 데이터 200개 불러오기
3. 모델 로드 (5분) - Qwen3-8B 모델을 메모리에 로드
4. 학습 (20-30분) - 실제 파인튜닝
5. GGUF 변환 (5분) - Ollama에서 쓸 수 있는 형식으로 변환
6. 저장 (1분) - Google Drive에 저장
```

### 시작하기 전 체크리스트
- [ ] Google 계정 로그인
- [ ] 런타임 → 런타임 유형 변경 → **T4 GPU** 선택
- [ ] combined_train.json 파일 준비 (Google Drive 또는 로컬)

# Qwen3-8B HR 도메인 파인튜닝

**환경**: Google Colab (T4 15GB / A100 40GB)

**목표**:
- SQL 생성 품질 향상
- 회사 규정 QA 품질 향상

**데이터셋**: 200개 (SQL 100 + RAG 100)

## 1. 환경 설정

In [ ]:
# ============================================================
# 📦 필수 라이브러리 설치
# ============================================================
# unsloth: 파인튜닝을 2배 빠르게, VRAM 70% 절약하는 라이브러리
# trl: Transformer Reinforcement Learning - 학습 도구
# peft: Parameter-Efficient Fine-Tuning - LoRA 등 효율적 학습 기법
# accelerate: GPU 가속화 라이브러리
# bitsandbytes: 4-bit/8-bit 양자화 (메모리 절약)
#
# %%capture: 설치 로그 숨기기 (깔끔한 출력)
# --no-deps: 의존성 충돌 방지
# ============================================================

%%capture
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# GPU 확인
!nvidia-smi

## 2. 데이터셋 업로드

GitHub에서 직접 다운로드하거나 파일 업로드

In [ ]:
# 방법 1: GitHub Raw URL에서 다운로드 (repo가 public인 경우)
# !wget -O combined_train.json "https://raw.githubusercontent.com/K-tuna/enterprise-hr-agent/main/data/finetuning/combined_train.json"

# 방법 2: Google Drive에서 마운트
from google.colab import drive
drive.mount('/content/drive')

# Drive에 combined_train.json 업로드 후 경로 지정
DATASET_PATH = "/content/drive/MyDrive/enterprise-hr-agent/combined_train.json"

# 방법 3: 직접 업로드
# from google.colab import files
# uploaded = files.upload()  # combined_train.json 선택
# DATASET_PATH = "combined_train.json"

In [ ]:
# 데이터셋 확인
import json

with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

print(f"총 데이터 수: {len(raw_data)}개")
print(f"\n첫 번째 예시:")
print(json.dumps(raw_data[0], ensure_ascii=False, indent=2))

## 3. 모델 로드

In [ ]:
# ============================================================
# 🤖 모델 로드
# ============================================================
# FastLanguageModel.from_pretrained(): 사전학습된 모델을 메모리에 로드
#
# 파라미터 설명:
# - model_name: 사용할 모델 ID
#   * "unsloth/Qwen3-8B" = Qwen3 8B 파라미터 버전 (Unsloth 최적화)
#   * 8B = 80억 개의 파라미터 (가중치)
#
# - max_seq_length: 최대 입력 길이
#   * 2048 토큰 ≈ 한글 약 1000자
#   * 길수록 긴 문맥 처리 가능, 메모리 더 사용
#
# - load_in_4bit: 4비트 양자화
#   * True: 32비트 → 4비트로 압축 (메모리 8배 절약!)
#   * 품질 손실 거의 없음
#
# - dtype: 데이터 타입
#   * None = 자동 감지 (GPU에 맞게 최적화)
#
# 결과물:
# - model: AI 모델 객체
# - tokenizer: 텍스트 ↔ 숫자 변환기
# ============================================================

from unsloth import FastLanguageModel
import torch

# 설정값
max_seq_length = 2048  # 최대 입력 길이 (토큰 단위)
lora_rank = 32         # LoRA 행렬 크기 (아래 셀에서 설명)

# 모델 로드 (4-bit 양자화)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-8B",  # Qwen3 8B 파라미터 모델
    max_seq_length=max_seq_length,
    load_in_4bit=True,              # 4비트 양자화로 메모리 절약
    dtype=None,                     # 자동 감지
)

print(f"✅ 모델 로드 완료: {model.config.model_type}")

In [ ]:
# ============================================================
# 🔧 LoRA 어댑터 설정
# ============================================================
# LoRA (Low-Rank Adaptation): 전체 모델이 아닌 일부만 학습하는 기술
#
# 왜 LoRA를 사용하나?
# - 전체 8B 파라미터 학습 → GPU 메모리 수백 GB 필요
# - LoRA: 0.1% 파라미터만 학습 → 16GB로 충분!
# - 원본 모델은 그대로 보존 (안전)
#
# 파라미터 설명:
# - r (rank): LoRA 행렬 크기
#   * 16: 가벼운 학습 (빠름, 메모리 적음)
#   * 32: 권장 (품질과 속도의 균형) ⭐
#   * 64: 고품질 (느림, 메모리 많이 필요)
#
# - target_modules: 학습할 레이어들
#   * q_proj, k_proj, v_proj: Attention의 Query, Key, Value
#   * o_proj: Attention 출력
#   * gate_proj, up_proj, down_proj: FFN (Feed-Forward Network)
#   → 이 7개 레이어만 학습해도 충분한 성능!
#
# - lora_alpha: 학습 강도 (보통 rank의 2배)
# - lora_dropout: 과적합 방지 (0 = 사용 안 함)
# - use_gradient_checkpointing: 메모리 절약 기법
# ============================================================

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,                    # LoRA rank (32 권장)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention 레이어
        "gate_proj", "up_proj", "down_proj",      # FFN 레이어
    ],
    lora_alpha=lora_rank * 2,       # 학습 강도 (rank의 2배)
    lora_dropout=0,                 # 과적합 방지 (0 = 사용 안 함)
    bias="none",                    # bias는 학습하지 않음
    use_gradient_checkpointing="unsloth",  # 메모리 절약
    random_state=42,                # 재현 가능한 결과
    max_seq_length=max_seq_length,
)

# 학습 가능한 파라미터 수 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"✅ 학습 가능 파라미터: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")

## 4. 데이터셋 준비

In [ ]:
# ============================================================
# 📊 데이터셋 준비
# ============================================================
# 모델이 이해할 수 있는 형식으로 데이터를 변환하는 과정
#
# 1. get_chat_template(): 모델별 대화 형식 템플릿 적용
#    - Qwen 모델은 <|im_start|>user ... <|im_end|> 형식 사용
#    - 모델마다 형식이 다름 (ChatGPT, Claude 등 각각 다름)
#
# 2. standardize_sharegpt(): 데이터 형식 표준화
#    - 입력: {"conversations": [{"role": "user", ...}, ...]}
#    - 다양한 데이터셋 형식을 통일된 형태로 변환
#
# 3. apply_chat_template(): 대화를 모델이 이해하는 텍스트로 변환
#    - 입력: [{"role": "user", "content": "질문"}]
#    - 출력: "<|im_start|>user\n질문<|im_end|>\n<|im_start|>assistant\n"
#
# 왜 이런 변환이 필요한가?
# - 모델은 단순 텍스트만 이해함
# - "user가 말했다", "assistant가 답했다"를 특수 토큰으로 구분
# ============================================================

from datasets import Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# 1. Tokenizer에 Qwen용 chat template 적용
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",  # Qwen3도 동일한 템플릿 사용
)

# 2. 포맷팅 함수 정의
def formatting_prompts_func(examples):
    """대화 데이터를 모델이 이해하는 텍스트로 변환"""
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, 
            tokenize=False,              # 텍스트로 반환 (토큰 아님)
            add_generation_prompt=False  # 학습용이므로 생성 프롬프트 불필요
        ) 
        for convo in convos
    ]
    return {"text": texts}

# 3. Dataset 생성 및 변환
dataset = Dataset.from_list(raw_data)           # 리스트 → Dataset 객체
dataset = standardize_sharegpt(dataset)         # 형식 표준화
dataset = dataset.map(formatting_prompts_func, batched=True)  # 변환 적용

# 결과 확인
print(f"✅ 데이터셋 준비 완료: {len(dataset)}개")
print(f"\n📝 포맷팅된 첫 번째 예시 (처음 500자):")
print(dataset[0]["text"][:500])

## 5. 학습

In [ ]:
# ============================================================
# 🎓 학습 설정
# ============================================================
# SFTTrainer: Supervised Fine-Tuning Trainer (지도학습 트레이너)
# - 정답이 있는 데이터로 모델을 학습시키는 도구
#
# 주요 파라미터 설명:
#
# [배치 관련]
# - per_device_train_batch_size (2): 한 번에 학습할 데이터 수
#   * 클수록 빠르지만 메모리 많이 사용
#   * 메모리 부족하면 1로 줄이기
#
# - gradient_accumulation_steps (4): 그래디언트 누적 횟수
#   * 실제 배치 크기 = batch_size × accumulation = 2×4 = 8
#   * 메모리 부족할 때: batch_size↓, accumulation↑
#
# [학습 관련]
# - num_train_epochs (3): 전체 데이터 반복 학습 횟수
#   * 1: 학습 부족할 수 있음
#   * 3: 권장 ⭐
#   * 5+: 과적합 위험 (외워버림)
#
# - learning_rate (2e-4 = 0.0002): 학습률
#   * 한 번에 얼마나 수정할지 결정
#   * 너무 크면: 학습 불안정 (튕김)
#   * 너무 작으면: 학습 느림
#   * LoRA 권장: 1e-4 ~ 3e-4
#
# - warmup_steps (10): 워밍업 구간
#   * 처음에 학습률을 천천히 올림 (안정성)
#
# [메모리 절약]
# - fp16/bf16: 반정밀도 학습 (메모리 절약, 속도 향상)
# - optim="adamw_8bit": 8비트 옵티마이저 (메모리 절약)
# - packing=True: 짧은 시퀀스들을 하나로 묶어 효율성 향상
# ============================================================

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",           # 학습할 텍스트 필드
    max_seq_length=max_seq_length,
    dataset_num_proc=2,                  # 데이터 처리 병렬화
    packing=True,                        # 짧은 시퀀스 패킹 (효율성↑)
    args=TrainingArguments(
        output_dir="./outputs",          # 체크포인트 저장 경로
        
        # 배치 설정
        per_device_train_batch_size=2,   # GPU당 배치 크기
        gradient_accumulation_steps=4,   # 그래디언트 누적 (실제 배치=8)
        
        # 학습 설정
        num_train_epochs=3,              # 전체 데이터 3회 반복
        learning_rate=2e-4,              # 학습률 (0.0002)
        warmup_steps=10,                 # 워밍업 구간
        
        # 로깅/저장
        logging_steps=10,                # 10스텝마다 로그 출력
        save_steps=100,                  # 100스텝마다 체크포인트 저장
        
        # 메모리 최적화
        fp16=not is_bfloat16_supported(),  # GPU가 bf16 미지원시 fp16 사용
        bf16=is_bfloat16_supported(),      # bf16 지원시 bf16 사용 (더 안정적)
        optim="adamw_8bit",              # 8비트 옵티마이저 (메모리 절약)
        
        # 기타
        weight_decay=0.01,               # 가중치 감쇠 (과적합 방지)
        lr_scheduler_type="linear",      # 학습률 스케줄러
        seed=42,                         # 재현 가능한 결과
        report_to="none",                # WandB 등 리포팅 비활성화
    ),
)

print("✅ 학습 설정 완료! 다음 셀에서 학습을 시작합니다.")

In [ ]:
# 학습 시작
print("=" * 50)
print("학습 시작...")
print("=" * 50)

trainer_stats = trainer.train()

print("\n학습 완료!")
print(f"총 학습 시간: {trainer_stats.metrics['train_runtime']:.2f}초")
print(f"최종 Loss: {trainer_stats.metrics['train_loss']:.4f}")

## 6. 테스트

In [ ]:
# 추론 모드 활성화
FastLanguageModel.for_inference(model)

# 테스트 질문
test_questions = [
    "개발팀 평균 급여는?",
    "연차휴가 며칠이야?",
    "김철수 평가 점수 알려줘",
    "재택근무 가능해?",
]

for question in test_questions:
    messages = [{"role": "user", "content": question}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
    )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-" * 50)

## 7. GGUF 변환 및 저장

In [ ]:
# ============================================================
# 💾 GGUF 변환
# ============================================================
# GGUF (GPT-Generated Unified Format)
# - llama.cpp에서 만든 모델 파일 형식
# - Ollama, LM Studio 등에서 사용하는 로컬 추론 표준
#
# 왜 GGUF로 변환하나?
# - PyTorch 모델 → Ollama에서 바로 사용 불가
# - GGUF로 변환해야 ollama run 명령어로 실행 가능
#
# quantization_method 옵션:
# - "q4_k_m": 4비트 양자화 (권장 ⭐)
#   * 파일 크기: ~5GB
#   * 품질: 좋음 (원본 대비 95%+)
#   * 속도: 빠름
#
# - "q8_0": 8비트 양자화
#   * 파일 크기: ~9GB
#   * 품질: 매우 좋음 (원본 대비 99%+)
#
# - "f16": 16비트 (양자화 없음)
#   * 파일 크기: ~16GB
#   * 품질: 원본 그대로
#
# ⚠️ 주의: Unsloth가 파일명을 자동 생성함 (예: qwen3-8b.Q4_K_M.gguf)
# ============================================================

import glob

# GGUF 변환
model.save_pretrained_gguf(
    "qwen3-hr",                      # 저장 디렉토리 이름
    tokenizer,
    quantization_method="q4_k_m",    # 4비트 양자화 (품질/크기 균형)
)

# 생성된 GGUF 파일 찾기 (Unsloth가 자동으로 파일명 생성)
gguf_files = glob.glob("*.gguf") + glob.glob("qwen3-hr/*.gguf")
print("✅ GGUF 변환 완료!")
print(f"\n📁 생성된 GGUF 파일: {gguf_files}")

# 첫 번째 GGUF 파일을 변수에 저장 (다음 셀에서 사용)
if gguf_files:
    GGUF_FILE = gguf_files[0]
    print(f"📦 사용할 파일: {GGUF_FILE}")
else:
    print("❌ GGUF 파일을 찾을 수 없습니다!")
    GGUF_FILE = None

!ls -lh *.gguf 2>/dev/null || ls -lh qwen3-hr/*.gguf 2>/dev/null || echo "파일 목록 확인 실패"

In [ ]:
# ============================================================
# 💾 Google Drive에 저장
# ============================================================
# 생성된 GGUF 파일을 Google Drive로 복사
# 로컬에서 다운로드하여 Ollama에 등록할 수 있음
# ============================================================

import shutil
import os

SAVE_DIR = "/content/drive/MyDrive/enterprise-hr-agent/models"
os.makedirs(SAVE_DIR, exist_ok=True)

if GGUF_FILE and os.path.exists(GGUF_FILE):
    # GGUF 파일을 qwen3-hr.gguf로 이름 변경하여 복사
    dest_path = os.path.join(SAVE_DIR, "qwen3-hr.gguf")
    shutil.copy(GGUF_FILE, dest_path)
    print(f"✅ 저장 완료!")
    print(f"   원본: {GGUF_FILE}")
    print(f"   저장: {dest_path}")
else:
    # 폴백: glob으로 다시 찾기
    import glob
    gguf_files = glob.glob("*.gguf") + glob.glob("qwen3-hr/*.gguf")
    if gguf_files:
        src = gguf_files[0]
        dest_path = os.path.join(SAVE_DIR, "qwen3-hr.gguf")
        shutil.copy(src, dest_path)
        print(f"✅ 저장 완료: {dest_path}")
    else:
        print("❌ GGUF 파일을 찾을 수 없습니다!")

print(f"\n📁 저장된 파일:")
!ls -lh "{SAVE_DIR}"

## 8. 로컬 배포 가이드

### Google Drive에서 다운로드
1. Google Drive에서 `qwen3-hr.gguf` 파일 다운로드
2. 프로젝트의 `models/` 디렉토리에 저장

### Ollama 등록
```bash
# models 디렉토리로 이동
cd models/

# Modelfile 생성
cat > Modelfile << 'EOF'
FROM ./qwen3-hr.gguf

TEMPLATE """{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
"""

PARAMETER temperature 0.1
PARAMETER stop "<|im_end|>"
EOF

# Ollama에 등록
ollama create qwen3-hr -f Modelfile

# 테스트
ollama run qwen3-hr "개발팀 평균 급여 SQL 작성해줘"
```

### 기존 모델과 비교
```bash
# 기존 모델
ollama run qwen3:8b "개발팀 평균 급여 SQL 작성해줘"

# 파인튜닝된 모델
ollama run qwen3-hr "개발팀 평균 급여 SQL 작성해줘"
```

In [ ]:
print("파인튜닝 완료!")
print("\n다음 단계:")
print("1. Google Drive에서 GGUF 파일 다운로드")
print("2. 로컬에서 Ollama에 등록")
print("3. 기존 qwen3:8b와 성능 비교")

## 📖 용어 사전

| 용어 | 설명 | 비유 |
|------|------|------|
| **파인튜닝** | 사전학습된 모델을 특정 데이터로 추가 학습 | 의대생 → 전문의 수련 |
| **LoRA** | 전체가 아닌 일부 파라미터만 학습하는 효율적 기법 | 전체 리모델링 대신 인테리어만 |
| **양자화** | 32비트 → 4/8비트로 압축 (메모리 절약) | 고화질 → 압축 동영상 |
| **토큰** | 텍스트의 최소 단위 | 대략 한글 0.5자, 영어 0.75단어 |
| **에폭 (Epoch)** | 전체 데이터를 1회 학습하는 단위 | 교과서 1회독 |
| **배치 (Batch)** | 한 번에 학습하는 데이터 묶음 | 한 번에 푸는 문제 수 |
| **학습률** | 학습 속도 (너무 크면 불안정, 작으면 느림) | 걸음 보폭 크기 |
| **GGUF** | Ollama/llama.cpp용 모델 파일 형식 | MP4 같은 동영상 포맷 |
| **VRAM** | GPU 메모리 | T4: 16GB, 2070 Super: 8GB |
| **Loss** | 모델의 오차 (낮을수록 좋음) | 시험 오답률 |
| **Attention** | 입력의 어느 부분에 집중할지 결정하는 메커니즘 | 밑줄 긋기 |
| **FFN** | Feed-Forward Network, 정보를 변환하는 레이어 | 정보 가공 공장 |

## ❓ 자주 발생하는 에러와 해결법

### 1. "CUDA out of memory" (GPU 메모리 부족)
**증상**: 학습 중 갑자기 멈추고 메모리 에러 발생

**해결 방법**:
```python
# 방법 1: 배치 크기 줄이기
per_device_train_batch_size=1  # 2 → 1로 변경

# 방법 2: 그래디언트 누적 늘리기
gradient_accumulation_steps=8  # 4 → 8로 변경

# 방법 3: 런타임 재시작
# 런타임 → 런타임 다시 시작 → 처음부터 다시 실행
```

---

### 2. "No GPU available" (GPU 없음)
**증상**: GPU를 찾을 수 없다는 에러

**해결 방법**:
1. 메뉴: `런타임` → `런타임 유형 변경`
2. 하드웨어 가속기: `T4 GPU` 선택
3. 저장 후 런타임 다시 시작

---

### 3. "File not found: combined_train.json"
**증상**: 데이터셋 파일을 찾을 수 없음

**해결 방법**:
1. Google Drive 마운트 확인 (왼쪽 파일 탭에서 drive 폴더 확인)
2. `DATASET_PATH` 변수가 실제 파일 경로와 일치하는지 확인
3. 직접 업로드 방법 사용:
```python
from google.colab import files
uploaded = files.upload()  # 파일 선택 창에서 combined_train.json 선택
DATASET_PATH = "combined_train.json"
```

---

### 4. 학습 중 세션 끊김
**증상**: 브라우저 탭을 오래 두면 연결 끊김

**원인**: Colab 정책 (90분 유휴 타임아웃 / 12시간 최대)

**해결 방법**:
- 브라우저 탭을 활성 상태로 유지
- 학습 중에는 자동으로 활성 상태 유지됨
- 끊겼을 경우: 체크포인트에서 재개 가능

---

### 5. "unsloth not found" (라이브러리 설치 실패)
**증상**: import unsloth 에러

**해결 방법**:
```python
# 설치 셀 다시 실행
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes

# 런타임 재시작 필요할 수 있음
```

---

### 6. Loss가 줄지 않거나 NaN 발생
**증상**: 학습 중 loss가 계속 같거나 NaN으로 표시

**해결 방법**:
```python
# 학습률 낮추기
learning_rate=1e-4  # 2e-4 → 1e-4

# 또는 warmup 늘리기
warmup_steps=20  # 10 → 20
```